In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as opt

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
SEED=1

In [3]:
random.seed(SEED)
torch.manual_seed(SEED)
#cudnn.deterministic = True

In [4]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [5]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

##########################
ARCH = 'resnet18' # set the architecture to RESNET 18
# please look up how to do that
########################
EPOCHS = 6
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 3e-5
PRINT_FREQ = 100
TRAIN_BATCH=256
VAL_BATCH=64
WORKERS=6

In [6]:
TRAINDIR="/home/user1/work/w251/imageNet-ILSVRC2012/download_and_prepare_imagenet_dataset/train"
VALDIR="/home/user1/work/w251/imageNet-ILSVRC2012/download_and_prepare_imagenet_dataset/val"


### Check if cuda is available here

In [7]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!

torch.cuda.is_available()

True

### Assign your GPU below

In [8]:
# Assign your GPU in this cell
GPU = 0

In [9]:
# set your active device to your GPU in this cell
if torch.cuda.is_available():  
  dev =  'cuda:{}'.format(GPU)
else:  
  dev = "cpu"  

device = torch.device(dev)

In [10]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [11]:
def train(train_loader, model, criterion, optimizer, scheduler, epoch, device=torch.device('cpu')):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.3e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        images = images.cuda(GPU, non_blocking=True)
        # send the target to cuda device
        target = target.cuda(GPU, non_blocking=True)
        
        
        # compute output
        output = model(images)

        # compute loss 
        loss = criterion(output, target)


        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        ## optimizer ..??
        optimizer.zero_grad()
        
        ## backprop!
        ### loss... ???
        loss.backward()
        
        # update the weights!
        ## optimier .. ??
        optimizer.step()
        scheduler.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [12]:
def validate(val_loader, model, criterion, device):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    # model ???
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)

            # compute loss
            loss  = criterion(output,target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [14]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [15]:
def save_checkpoint(state, is_best, filename='./checkpoint.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, './model_best.pth.tar')

In [16]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [17]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [18]:
IMG_SIZE = 32
IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [19]:
# select the model
print("=> Creating model '{}'".format(ARCH))
model = models.__dict__[ARCH]()

=> Creating model 'resnet18'


### Send the model to the cuda device

In [20]:
# send the model to the cuda device..
model = model.to(device)

### Instantiate the loss to cross entropy

In [21]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [22]:
# use SGD .. use the momentum and weight decay vars
optimizer = opt.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [23]:
# use CosineAnnealingLR
#scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
# Use Super Convergence                                                                           
#scheduler = OneCycleLR(optimizer, max_lr=1.0, steps_per_epoch=len(train_loader), epochs=EPOCHS)

### Create the train dataset object

In [24]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),# padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])


In [25]:
train_dataset = torchvision.datasets.ImageFolder(TRAINDIR, transform=transform_train)

### Create the val dataset object

In [26]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [27]:
val_dataset = torchvision.datasets.ImageFolder(VALDIR, transform=transform_val)

### Create the train dataloader

In [28]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=256, num_workers=WORKERS, shuffle=True)
#train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=128, shuffle=True)

### Create the c

In [29]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset,  batch_size=64, num_workers=WORKERS, shuffle=True) 
#val_loader = torch.utils.data.DataLoader(val_dataset,  batch_size=128, shuffle=True) 

In [30]:
best_acc1 = 0
# Use Super Convergence                                                                           
scheduler = OneCycleLR(optimizer, max_lr=1.0, steps_per_epoch=len(train_loader), epochs=EPOCHS)

In [31]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, scheduler, epoch, device)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, device)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
        'scheduler' : scheduler.state_dict(),
    }, is_best)
    
    #scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][   0/5005]	Time 33.021 (33.021)	Data 23.073 (23.073)	Loss 7.060e+00 (7.060e+00)	Acc@1   0.39 (  0.39)	Acc@5   0.78 (  0.78)
Epoch: [0][ 100/5005]	Time  0.817 ( 3.044)	Data  0.095 ( 2.170)	Loss 6.790e+00 (6.957e+00)	Acc@1   0.39 (  0.25)	Acc@5   0.78 (  1.12)
Epoch: [0][ 200/5005]	Time  1.106 ( 2.905)	Data  0.055 ( 2.078)	Loss 6.552e+00 (6.830e+00)	Acc@1   0.39 (  0.35)	Acc@5   3.52 (  1.63)
Epoch: [0][ 300/5005]	Time  2.960 ( 2.850)	Data  2.330 ( 2.026)	Loss 6.525e+00 (6.725e+00)	Acc@1   0.78 (  0.55)	Acc@5   1.95 (  2.13)
Epoch: [0][ 400/5005]	Time  2.023 ( 2.826)	Data  1.327 ( 1.999)	Loss 6.260e+00 (6.629e+00)	Acc@1   1.95 (  0.71)	Acc@5   2.73 (  2.78)
Epoch: [0][ 500/5005]	Time  0.503 ( 2.801)	Data  0.037 ( 1.981)	Loss 6.148e+00 (6.541e+00)	Acc@1   1.56 (  0.89)	Acc@5   6.25 (  3.39)
Epoch: [0][ 600/5005]	Time  2.231 ( 2.794)	Data  1.557 ( 1.981)	Loss 5.786e+00 (6.459e+00)	Acc@1   2.34 (  1.09)	Acc@5  10.55 (  4.01)
Epoch: [0][ 700/5005]	Time  0.741 ( 2.793)	Data  0.048 

Epoch: [1][ 300/5005]	Time  1.366 ( 2.312)	Data  0.129 ( 1.127)	Loss 3.919e+00 (3.884e+00)	Acc@1  21.88 ( 22.50)	Acc@5  42.97 ( 45.21)
Epoch: [1][ 400/5005]	Time  1.238 ( 2.302)	Data  0.098 ( 1.127)	Loss 3.882e+00 (3.873e+00)	Acc@1  23.05 ( 22.66)	Acc@5  45.70 ( 45.41)
Epoch: [1][ 500/5005]	Time  1.446 ( 2.302)	Data  0.111 ( 1.135)	Loss 4.093e+00 (3.865e+00)	Acc@1  18.75 ( 22.81)	Acc@5  40.62 ( 45.56)
Epoch: [1][ 600/5005]	Time  3.560 ( 2.303)	Data  2.488 ( 1.145)	Loss 3.867e+00 (3.862e+00)	Acc@1  23.05 ( 22.84)	Acc@5  45.31 ( 45.61)
Epoch: [1][ 700/5005]	Time  2.752 ( 2.302)	Data  1.667 ( 1.155)	Loss 3.919e+00 (3.853e+00)	Acc@1  22.66 ( 23.01)	Acc@5  44.92 ( 45.82)
Epoch: [1][ 800/5005]	Time  2.131 ( 2.303)	Data  0.992 ( 1.166)	Loss 3.689e+00 (3.843e+00)	Acc@1  26.17 ( 23.18)	Acc@5  48.44 ( 45.99)
Epoch: [1][ 900/5005]	Time  2.051 ( 2.306)	Data  1.048 ( 1.176)	Loss 3.418e+00 (3.834e+00)	Acc@1  29.30 ( 23.29)	Acc@5  53.12 ( 46.18)
Epoch: [1][1000/5005]	Time  1.290 ( 2.313)	Data  0.176 

Epoch: [2][ 600/5005]	Time  2.668 ( 2.313)	Data  1.430 ( 1.160)	Loss 3.530e+00 (3.272e+00)	Acc@1  28.12 ( 31.81)	Acc@5  51.56 ( 56.66)
Epoch: [2][ 700/5005]	Time  5.588 ( 2.316)	Data  4.480 ( 1.170)	Loss 3.445e+00 (3.271e+00)	Acc@1  29.69 ( 31.80)	Acc@5  53.12 ( 56.69)
Epoch: [2][ 800/5005]	Time  1.924 ( 2.329)	Data  0.253 ( 1.186)	Loss 3.248e+00 (3.273e+00)	Acc@1  35.55 ( 31.77)	Acc@5  57.81 ( 56.68)
Epoch: [2][ 900/5005]	Time  2.586 ( 2.353)	Data  1.305 ( 1.196)	Loss 3.233e+00 (3.272e+00)	Acc@1  32.42 ( 31.82)	Acc@5  55.86 ( 56.72)
Epoch: [2][1000/5005]	Time  1.501 ( 2.368)	Data  0.186 ( 1.208)	Loss 3.316e+00 (3.272e+00)	Acc@1  29.69 ( 31.81)	Acc@5  58.20 ( 56.73)
Epoch: [2][1100/5005]	Time  1.218 ( 2.369)	Data  0.195 ( 1.222)	Loss 3.079e+00 (3.271e+00)	Acc@1  35.55 ( 31.83)	Acc@5  62.50 ( 56.76)
Epoch: [2][1200/5005]	Time  3.602 ( 2.373)	Data  2.546 ( 1.239)	Loss 3.481e+00 (3.271e+00)	Acc@1  26.56 ( 31.84)	Acc@5  52.34 ( 56.73)
Epoch: [2][1300/5005]	Time  1.271 ( 2.376)	Data  0.308 

Epoch: [3][ 900/5005]	Time  3.109 ( 2.359)	Data  1.874 ( 1.184)	Loss 3.095e+00 (3.032e+00)	Acc@1  33.59 ( 35.68)	Acc@5  59.38 ( 61.03)
Epoch: [3][1000/5005]	Time  1.300 ( 2.363)	Data  0.082 ( 1.198)	Loss 3.072e+00 (3.031e+00)	Acc@1  37.50 ( 35.70)	Acc@5  59.38 ( 61.06)
Epoch: [3][1100/5005]	Time  3.738 ( 2.368)	Data  2.625 ( 1.215)	Loss 2.941e+00 (3.029e+00)	Acc@1  35.16 ( 35.74)	Acc@5  63.67 ( 61.09)
Epoch: [3][1200/5005]	Time  2.166 ( 2.369)	Data  1.033 ( 1.229)	Loss 3.024e+00 (3.031e+00)	Acc@1  39.06 ( 35.71)	Acc@5  61.33 ( 61.07)
Epoch: [3][1300/5005]	Time  6.913 ( 2.377)	Data  5.722 ( 1.249)	Loss 2.938e+00 (3.028e+00)	Acc@1  39.45 ( 35.79)	Acc@5  62.50 ( 61.12)
Epoch: [3][1400/5005]	Time  2.170 ( 2.380)	Data  1.366 ( 1.262)	Loss 2.936e+00 (3.026e+00)	Acc@1  41.41 ( 35.83)	Acc@5  61.72 ( 61.15)
Epoch: [3][1500/5005]	Time  4.710 ( 2.388)	Data  3.695 ( 1.283)	Loss 2.921e+00 (3.024e+00)	Acc@1  34.38 ( 35.86)	Acc@5  62.11 ( 61.19)
Epoch: [3][1600/5005]	Time  1.440 ( 2.395)	Data  0.766 

Epoch: [4][1200/5005]	Time  1.421 ( 2.342)	Data  0.235 ( 1.219)	Loss 2.888e+00 (2.782e+00)	Acc@1  38.67 ( 40.00)	Acc@5  62.11 ( 65.21)
Epoch: [4][1300/5005]	Time  1.068 ( 2.348)	Data  0.085 ( 1.233)	Loss 2.855e+00 (2.779e+00)	Acc@1  33.20 ( 40.06)	Acc@5  64.45 ( 65.26)
Epoch: [4][1400/5005]	Time  3.258 ( 2.357)	Data  2.581 ( 1.253)	Loss 2.954e+00 (2.778e+00)	Acc@1  32.81 ( 40.10)	Acc@5  60.55 ( 65.25)
Epoch: [4][1500/5005]	Time  4.008 ( 2.368)	Data  3.079 ( 1.273)	Loss 2.686e+00 (2.775e+00)	Acc@1  40.62 ( 40.13)	Acc@5  65.62 ( 65.30)
Epoch: [4][1600/5005]	Time  7.356 ( 2.378)	Data  6.619 ( 1.296)	Loss 2.685e+00 (2.772e+00)	Acc@1  39.84 ( 40.18)	Acc@5  68.75 ( 65.36)
Epoch: [4][1700/5005]	Time  2.482 ( 2.384)	Data  1.885 ( 1.315)	Loss 2.886e+00 (2.770e+00)	Acc@1  39.06 ( 40.23)	Acc@5  64.06 ( 65.39)
Epoch: [4][1800/5005]	Time  5.341 ( 2.398)	Data  4.473 ( 1.338)	Loss 2.695e+00 (2.767e+00)	Acc@1  42.97 ( 40.28)	Acc@5  66.80 ( 65.43)
Epoch: [4][1900/5005]	Time  4.217 ( 2.408)	Data  3.122 

Epoch: [5][1500/5005]	Time  1.430 ( 2.381)	Data  0.247 ( 1.333)	Loss 2.309e+00 (2.314e+00)	Acc@1  46.88 ( 48.55)	Acc@5  74.61 ( 72.85)
Epoch: [5][1600/5005]	Time  7.641 ( 2.394)	Data  6.969 ( 1.354)	Loss 2.315e+00 (2.308e+00)	Acc@1  48.44 ( 48.67)	Acc@5  73.44 ( 72.93)
Epoch: [5][1700/5005]	Time  1.690 ( 2.404)	Data  0.775 ( 1.373)	Loss 2.095e+00 (2.303e+00)	Acc@1  53.91 ( 48.80)	Acc@5  75.00 ( 73.00)
Epoch: [5][1800/5005]	Time  0.864 ( 2.414)	Data  0.164 ( 1.391)	Loss 2.226e+00 (2.296e+00)	Acc@1  50.78 ( 48.92)	Acc@5  75.39 ( 73.10)
Epoch: [5][1900/5005]	Time  2.658 ( 2.421)	Data  1.875 ( 1.407)	Loss 2.220e+00 (2.291e+00)	Acc@1  54.69 ( 49.06)	Acc@5  71.88 ( 73.20)
Epoch: [5][2000/5005]	Time  1.254 ( 2.431)	Data  0.512 ( 1.424)	Loss 2.313e+00 (2.284e+00)	Acc@1  47.27 ( 49.17)	Acc@5  71.48 ( 73.30)
Epoch: [5][2100/5005]	Time  1.157 ( 2.440)	Data  0.216 ( 1.437)	Loss 2.203e+00 (2.279e+00)	Acc@1  48.83 ( 49.26)	Acc@5  74.22 ( 73.37)
Epoch: [5][2200/5005]	Time  1.041 ( 2.445)	Data  0.139 

In [32]:
#from torchsummary import summary
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  